# Covid-19 Open Vaccine

**Name: Stuart Hopkins**

**A-Number: A02080107**


## Introduction
Winning the fight against the COVID-19 pandemic will require an effective vaccine that can be equitably and widely distributed. Building upon decades of research has allowed scientists to accelerate the search for a vaccine against COVID-19, but every day that goes by without a vaccine has enormous costs for the world nonetheless. We need new, fresh ideas from all corners of the world. Could online gaming and crowdsourcing help solve a worldwide pandemic? Pairing scientific and crowdsourced intelligence could help computational biochemists make measurable progress.

mRNA vaccines have taken the lead as the fastest vaccine candidates for COVID-19, but currently, they face key potential limitations. One of the biggest challenges right now is how to design super stable messenger RNA molecules (mRNA). Conventional vaccines (like your seasonal flu shots) are packaged in disposable syringes and shipped under refrigeration around the world, but that is not currently possible for mRNA vaccines.

Researchers have observed that RNA molecules have the tendency to spontaneously degrade. This is a serious limitation--a single cut can render the mRNA vaccine useless. Currently, little is known on the details of where in the backbone of a given RNA is most prone to being affected. Without this knowledge, current mRNA vaccines against COVID-19 must be prepared and shipped under intense refrigeration, and are unlikely to reach more than a tiny fraction of human beings on the planet unless they can be stabilized.



The Eterna community, led by Professor Rhiju Das, a computational biochemist at Stanford’s School of Medicine, brings together scientists and gamers to solve puzzles and invent medicine. Eterna is an online video game platform that challenges players to solve scientific problems such as mRNA design through puzzles. The solutions are synthesized and experimentally tested at Stanford by researchers to gain new insights about RNA molecules. The Eterna community has previously unlocked new scientific principles, made new diagnostics against deadly diseases, and engaged the world’s most potent intellectual resources for the betterment of the public. The Eterna community has advanced biotechnology through its contribution in over 20 publications, including advances in RNA biotechnology.

In this competition, we are looking to leverage the data science expertise of the Kaggle community to develop models and design rules for RNA degradation. Your model will predict likely degradation rates at each base of an RNA molecule, trained on a subset of an Eterna dataset comprising over 3000 RNA molecules (which span a panoply of sequences and structures) and their degradation rates at each position. We will then score your models on a second generation of RNA sequences that have just been devised by Eterna players for COVID-19 mRNA vaccines. These final test sequences are currently being synthesized and experimentally characterized at Stanford University in parallel to your modeling efforts -- Nature will score your models!

Improving the stability of mRNA vaccines was a problem that was being explored before the pandemic but was expected to take many years to solve. Now, we must solve this deep scientific challenge in months, if not weeks, to accelerate mRNA vaccine research and deliver a refrigerator-stable vaccine against SARS-CoV-2, the virus behind COVID-19. The problem we are trying to solve has eluded academic labs, industry R&D groups, and supercomputers, and so we are turning to you. To help, you can join the team of video game players, scientists, and developers at Eterna to unlock the key in our fight against this devastating pandemic.

## Resources
Big thanks to @xhlulu for the base to this project. The submission is pulled from many sources (and personal code of course), but is largely based upon the workbook by @xhlulu. 

## Dependancies
#### Required Files:
All required files can be downloaded <a href="https://www.kaggle.com/c/stanford-covid-vaccine/data">here</a>
- input/test.json
- input/train.json
- input/sample_submission.csv

In [ ]:
### Uncomment packages that you do not currently have installed to install them
# !pip install pandas
# !pip install numpy
# !pip install plotly
# !pip install tensorflow
# !pip install sklearn

In [ ]:
import json

import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow.keras.layers as L
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Set random seed
Here we are setting the seed so that we can get the same random numbers every time.





In [ ]:
tf.random.set_seed(42069)
np.random.seed(42069)

## Load and Preprocess Data

In [ ]:
bs = 32
pred_len = 68
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [ ]:
y_true = tf.random.normal((bs, pred_len, len(pred_cols)))
y_pred = tf.random.normal((bs, pred_len, len(pred_cols)))

In [ ]:
data_dir = 'input/'
train = pd.read_json(data_dir + 'train.json', lines=True)
test = pd.read_json(data_dir + 'test.json', lines=True)
sample_df = pd.read_csv(data_dir + 'sample_submission.csv')

In [ ]:
train = train.query("signal_to_noise >= 1")

In [ ]:
def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )

In [ ]:
def preprocess_inputs(df, token2int, cols=["sequence", "structure", "predicted_loop_type"]):
    return pandas_list_to_array(
        df[cols].applymap(lambda seq: [token2int[x] for x in seq])
    )

In [ ]:
# Map each character to an integer
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

train_labels = pandas_list_to_array(train[pred_cols])
train_inputs = preprocess_inputs(train, token2int)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    train_inputs, train_labels, test_size=.1, random_state=34, stratify=train.SN_filter)

In [ ]:
public_df = test.query("seq_length == 107")
private_df = test.query("seq_length == 130")

public_inputs = preprocess_inputs(public_df, token2int)
private_inputs = preprocess_inputs(private_df, token2int)

## Build and train model
We will train a bi-directional GRU model. It has three layer and has dropout. To learn more about RNNs, LSTM and GRU, please see this blog post.

In [ ]:
def MCRMSE(y_true, y_pred):
    column_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(column_mse), axis=1)

In [ ]:
def lstm_layer(hidden_dim, dropout):
    return L.Bidirectional(L.LSTM(
        hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))

In [ ]:
def build_model(embed_size, seq_len=107, pred_len=68, dropout=0.5, 
                sp_dropout=0.2, embed_dim=200, hidden_dim=256, n_layers=3):
    inputs = L.Input(shape=(seq_len, 3))
    embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    hidden = L.SpatialDropout1D(sp_dropout)(reshaped)
    
    for x in range(n_layers):
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    # Since we are only making predictions on the first part of each sequence, 
    # we have to truncate it
    truncated = hidden[:, :pred_len]
    out = L.Dense(5, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.optimizers.Adam(), loss=MCRMSE)
    
    return model

In [ ]:
model = build_model(embed_size=len(token2int))
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 107, 3)]          0         
_________________________________________________________________
embedding (Embedding)        (None, 107, 3, 200)       2800      
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 107, 600)]        0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 107, 600)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 107, 512)          1755136   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 107, 512)          1574912   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 107, 512)         

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=64,
    epochs=80, # TODO: Up this number to 80
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
)

Epoch 1/80
30/30 - 219s - loss: 0.5151 - val_loss: 0.4171
Epoch 2/80
30/30 - 212s - loss: 0.4046 - val_loss: 0.3764
Epoch 3/80
30/30 - 211s - loss: 0.3823 - val_loss: 0.3632
Epoch 4/80
30/30 - 221s - loss: 0.3661 - val_loss: 0.3435
Epoch 5/80
30/30 - 217s - loss: 0.3511 - val_loss: 0.3347
Epoch 6/80
30/30 - 215s - loss: 0.3410 - val_loss: 0.3269
Epoch 7/80
30/30 - 215s - loss: 0.3336 - val_loss: 0.3208
Epoch 8/80
30/30 - 215s - loss: 0.3266 - val_loss: 0.3158
Epoch 9/80
30/30 - 215s - loss: 0.3207 - val_loss: 0.3040
Epoch 10/80
30/30 - 214s - loss: 0.3104 - val_loss: 0.2960
Epoch 11/80
30/30 - 221s - loss: 0.3053 - val_loss: 0.2911
Epoch 12/80
30/30 - 215s - loss: 0.2987 - val_loss: 0.2854
Epoch 13/80
30/30 - 217s - loss: 0.2913 - val_loss: 0.2803
Epoch 14/80
30/30 - 220s - loss: 0.2853 - val_loss: 0.2704
Epoch 15/80
30/30 - 216s - loss: 0.2764 - val_loss: 0.2641
Epoch 16/80
30/30 - 214s - loss: 0.2688 - val_loss: 0.2589
Epoch 17/80
30/30 - 213s - loss: 0.2643 - val_loss: 0.2603
Epoch 

In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'MCRMSE'}, 
    title='Training History')
fig.show()

## Load models and make predictions
Public and private sets have different sequence lengths, so we will preprocess them separately and load models of different tensor shapes. This is possible because RNN models can accept sequences of varying lengths as inputs.

In [ ]:
# Caveat: The prediction format requires the output to be the same length as the input,
# although it's not the case for the training data.
model_public = build_model(seq_len=107, pred_len=107, embed_size=len(token2int))
model_private = build_model(seq_len=130, pred_len=130, embed_size=len(token2int))

model_public.load_weights('model.h5')
model_private.load_weights('model.h5')

In [ ]:
public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

## Post-processing and submit
For each sample, we take the predicted tensors of shape (107, 5) or (130, 5), and convert them to the long format (i.e.  629×107,5  or  3005×130,5):

In [ ]:
preds_ls = []

for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.673217,0.653089,0.511145,2.026246,0.749455,id_00073f8be_0
1,1.967305,2.758735,2.942523,3.896030,2.698194,id_00073f8be_1
2,1.375278,0.553981,0.659123,0.651248,0.662396,id_00073f8be_2
3,1.215842,1.120134,1.583379,1.181685,1.647009,id_00073f8be_3
4,0.852111,0.659991,0.877378,0.589644,0.921785,id_00073f8be_4


In [ ]:
submission = sample_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False) 